In [7]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import shutil
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from contour_features import load_image, process_image, cut_section, cut_image, calculate_features, show_process
from time import time

In [2]:
seed = 2802
test_size = 0.2
batch_size = 4
num_classes = 3

input_dir = "data/images"
target_dir = "data/mask"

input_shape = (320,320)

features = ['dist', 'extent', 'slope', 'pwp_limit', 'pwp_middle']

In [4]:
df = pd.read_csv('data/dataset_train_test.csv') 
df['path'] = input_dir+'/'+df['image_name']+'.jpg'
df['category'] = np.where(df['class']==1, 'NB', np.where(df['class']==2, 'NB', 'B'))
df['binary_class'] = np.where(df['class']==1, 0, np.where(df['class']==2, 0, 1))

df.head()

,Unnamed: 0,image_name,part,section,min_height,max_height,diff_height,piece_position,class,image_id,set,path,category,binary_class
0,0,P01_001_0,P01,0,181,200,19,Down,2,1,Train,data/images/P01_001_0.jpg,NB,0
1,1,P01_001_1,P01,1,198,208,10,Down,1,1,Train,data/images/P01_001_1.jpg,NB,0
2,2,P01_001_2,P01,2,204,214,10,Down,1,1,Train,data/images/P01_001_2.jpg,NB,0
3,3,P01_001_3,P01,3,200,212,12,Down,1,1,Train,data/images/P01_001_3.jpg,NB,0
4,4,P01_002_0,P01,0,232,238,6,Down,1,2,Train,data/images/P01_002_0.jpg,NB,0


In [5]:
data = pd.DataFrame(columns=['img_name', 'category','set']+features)
for _, row in df.iterrows():
    print(row['path'])
    image = load_image(row['path'])
    img_process = process_image(image)
    data_vector =  cut_section(img_process[-1])
    if "dist" in data_vector:  
            img_cut = cut_image(img_process[0],data_vector['y1'],data_vector['y2'])
            if img_cut.shape[0]>10:
                img_process_cut = process_image(img_cut, clahe_tile = (5,5), kernel_size=(17,39))            
                data_vector_cut =  cut_section(img_process_cut[-1], True, True)
                
                if "contour_line" in data_vector_cut:
                    calculate_features(img_process_cut[-1], data_vector_cut, True)  
                    if "extent" in data_vector_cut:
                        info = {'img_name': row['image_name'],'category': row['class'], 'binary_class': row['binary_class'], 'set': row['set']}
                        for feature in features:
                            info[feature] = data_vector_cut[feature]
                        data = data.append(info, ignore_index=True)
data.to_csv('results/traditional_data_complete.csv', index=False, sep=';', decimal=".")
data.head()

data/images/P01_001_0.jpg
data/images/P01_001_1.jpg
data/images/P01_001_2.jpg
data/images/P01_001_3.jpg
data/images/P01_002_0.jpg
data/images/P01_002_1.jpg
data/images/P01_002_2.jpg
data/images/P01_002_3.jpg
data/images/P01_003_0.jpg
data/images/P01_003_1.jpg
data/images/P01_003_2.jpg
data/images/P01_003_3.jpg
data/images/P01_004_0.jpg
data/images/P01_004_1.jpg
data/images/P01_004_2.jpg
data/images/P01_004_3.jpg
data/images/P01_005_0.jpg
data/images/P01_005_1.jpg
data/images/P01_005_2.jpg
data/images/P01_005_3.jpg
data/images/P01_006_0.jpg
data/images/P01_006_1.jpg
data/images/P01_006_2.jpg
data/images/P01_006_3.jpg
data/images/P01_007_0.jpg
data/images/P01_007_1.jpg
data/images/P01_007_2.jpg
data/images/P01_007_3.jpg
data/images/P01_008_0.jpg
data/images/P01_008_1.jpg
data/images/P01_008_2.jpg
data/images/P01_008_3.jpg
data/images/P01_009_0.jpg
data/images/P01_009_1.jpg
data/images/P01_009_2.jpg
data/images/P01_009_3.jpg
data/images/P01_010_0.jpg
data/images/P01_010_1.jpg
data/images/

data/images/P03_080_1.jpg
data/images/P03_080_2.jpg
data/images/P03_080_3.jpg
data/images/P03_081_0.jpg
data/images/P03_081_1.jpg
data/images/P03_081_2.jpg
data/images/P03_081_3.jpg
data/images/P03_082_0.jpg
data/images/P03_082_1.jpg
data/images/P03_082_2.jpg
data/images/P03_082_3.jpg
data/images/P03_083_0.jpg
data/images/P03_083_1.jpg
data/images/P03_083_2.jpg
data/images/P03_083_3.jpg
data/images/P03_084_0.jpg
data/images/P03_084_1.jpg
data/images/P03_084_2.jpg
data/images/P03_084_3.jpg
data/images/P03_085_0.jpg
data/images/P03_085_1.jpg
data/images/P03_085_2.jpg
data/images/P03_085_3.jpg
data/images/P03_086_0.jpg
data/images/P03_086_1.jpg
data/images/P03_086_2.jpg
data/images/P03_086_3.jpg
data/images/P03_087_0.jpg
data/images/P03_087_1.jpg
data/images/P03_087_2.jpg
data/images/P03_087_3.jpg
data/images/P03_088_0.jpg
data/images/P03_088_1.jpg
data/images/P03_088_2.jpg
data/images/P03_088_3.jpg
data/images/P03_089_0.jpg
data/images/P03_089_1.jpg
data/images/P03_089_2.jpg
data/images/

data/images/P12_159_1.jpg
data/images/P12_159_2.jpg
data/images/P12_159_3.jpg
data/images/P12_160_0.jpg
data/images/P12_160_1.jpg
data/images/P12_160_2.jpg
data/images/P12_160_3.jpg
data/images/P12_161_0.jpg
data/images/P12_161_1.jpg
data/images/P12_161_2.jpg
data/images/P12_161_3.jpg
data/images/P12_162_0.jpg
data/images/P12_162_1.jpg
data/images/P12_162_2.jpg
data/images/P12_162_3.jpg
data/images/P12_163_0.jpg
data/images/P12_163_1.jpg
data/images/P12_163_2.jpg
data/images/P12_163_3.jpg
data/images/P12_164_0.jpg
data/images/P12_164_1.jpg
data/images/P12_164_2.jpg
data/images/P12_164_3.jpg
data/images/P12_165_0.jpg
data/images/P12_165_1.jpg
data/images/P12_165_2.jpg
data/images/P12_165_3.jpg
data/images/P12_166_0.jpg
data/images/P12_166_1.jpg
data/images/P12_166_2.jpg
data/images/P12_166_3.jpg
data/images/P12_167_0.jpg
data/images/P12_167_1.jpg
data/images/P12_167_2.jpg
data/images/P12_167_3.jpg
data/images/P12_168_0.jpg
data/images/P12_168_1.jpg
data/images/P12_168_2.jpg
data/images/

data/images/P03_117_3.jpg
data/images/P03_071_4.jpg
data/images/P03_118_3.jpg
data/images/P03_119_3.jpg
data/images/P03_120_3.jpg
data/images/P03_121_3.jpg
data/images/P03_122_3.jpg
data/images/P03_123_3.jpg
data/images/P03_124_3.jpg
data/images/P03_125_3.jpg
data/images/P03_126_3.jpg
data/images/P03_127_3.jpg
data/images/P03_128_3.jpg
data/images/P03_129_3.jpg
data/images/P03_130_3.jpg
data/images/P03_131_3.jpg
data/images/P03_132_3.jpg
data/images/P03_133_3.jpg
data/images/P03_080_4.jpg
data/images/P03_081_4.jpg
data/images/P03_082_4.jpg
data/images/P03_134_3.jpg
data/images/P03_135_3.jpg
data/images/P03_136_3.jpg
data/images/P03_086_4.jpg
data/images/P03_137_3.jpg
data/images/P03_138_3.jpg
data/images/P03_139_3.jpg
data/images/P03_096_4.jpg
data/images/P03_097_4.jpg
data/images/P03_098_4.jpg
data/images/P03_099_4.jpg
data/images/P03_100_4.jpg
data/images/P03_101_4.jpg
data/images/P03_102_4.jpg
data/images/P03_103_4.jpg
data/images/P03_104_4.jpg
data/images/P05_105_4.jpg
data/images/

,img_name,category,set,dist,extent,slope,pwp_limit,pwp_middle,binary_class
0,P01_001_0,2,Train,18,0.668421,-0.066686,0.594729,0.558675,0.0
1,P01_001_1,1,Train,9,0.477031,-0.019870,0.623259,0.620682,0.0
2,P01_001_2,1,Train,9,0.472813,0.004289,0.583283,0.578049,0.0
3,P01_001_3,1,Train,11,0.483594,-0.012722,0.507450,0.499777,0.0
4,P01_002_0,1,Train,4,0.325625,-0.006312,0.600273,0.599306,0.0


In [6]:
data = pd.read_csv('results/traditional_data_complete.csv', delimiter=';', decimal=".")

In [8]:
data_train = data[data['set']=='Train']
data_test = data[data['set']=='Test']

X_train = data_train[features]
y_train = data_train[['binary_class']].values.ravel()

X_test = data_test[features]
y_test = data_test[['binary_class']].values.ravel()

In [9]:
model = RandomForestClassifier(random_state=seed,n_jobs=-1, criterion = 'entropy', max_depth= 14, n_estimators=95)
start = time()
model = model.fit(X_train, y_train)
y_predict_train = model.predict(X_train)
print(classification_report(y_train,y_predict_train, digits=4))
y_predict= model.predict(X_test)
print(classification_report(y_test,y_predict, digits=4))
train_time = time()-start
print(train_time)

              precision    recall  f1-score   support

         0.0     1.0000    1.0000    1.0000       661
         1.0     1.0000    1.0000    1.0000       185

    accuracy                         1.0000       846
   macro avg     1.0000    1.0000    1.0000       846
weighted avg     1.0000    1.0000    1.0000       846

              precision    recall  f1-score   support

         0.0     0.9162    0.9053    0.9107       169
         1.0     0.6522    0.6818    0.6667        44

    accuracy                         0.8592       213
   macro avg     0.7842    0.7936    0.7887       213
weighted avg     0.8616    0.8592    0.8603       213

0.5119109153747559
